# 🤖 ATENA-TF Live Recommendations - Colab Edition

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Eden-Mironi/Atena-TF/blob/main/Notebooks/Live_Recommendations_System_Colab.ipynb)

**Interactive AI-Powered Data Exploration**

Click "Runtime" → "Run all" to start!

---

## How it works:
1. 🧠 Trained model suggests next best action
2. 👤 You decide: follow AI or choose your own
3. 📊 See instant results
4. 🔄 Repeat and explore!

In [ ]:
# Setup
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f"Running in {'Colab' if IN_COLAB else 'local'} mode")

In [ ]:
if IN_COLAB:
    %pip install -q tensorflow>=2.16.0 tensorflow-probability>=0.24.0
    %pip install -q gym==0.25.2 'numpy<2.0.0' pandas matplotlib seaborn scipy
    %pip install -q cachetools zss nltk openpyxl ipywidgets
    print("✅ Dependencies installed!")

In [ ]:
if IN_COLAB:
    import os
    
    # Clone from GitHub
    if not os.path.exists('/content/atena-tf'):
        print("📥 Cloning from GitHub...")
        !git clone https://github.com/Eden-Mironi/Atena-TF.git /content/atena-tf
        print("✅ Cloned!")
    
    %cd /content/atena-tf
    %pip install -q -e .
    print("✅ Ready!")

---

## Initialize Recommender System

In [ ]:
# Import recommender components
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np

if not hasattr(np, "bool8"):
    np.bool8 = np.bool_

from live_recommender_agent import TFRecommenderAgent, find_latest_trained_model

print("✅ Imports successful!")

In [ ]:
# Load trained model
model_path = find_latest_trained_model()

if model_path:
    print(f"✅ Found model: {model_path}")
    
    recommender_agent = TFRecommenderAgent(
        model_path=model_path,
        dataset_number=0,
        schema='NETWORKING'
    )
    
    print("\n🎉 Recommender initialized!")
    print(f"📊 Dataset shape: {recommender_agent.original_dataset.shape}")
    print(f"📋 Columns: {list(recommender_agent.original_dataset.columns)}")
else:
    print("⚠️ No trained model found")
    print("Please train first: python main.py")

## 🎛️ Interactive Control Panel

In [ ]:
# Create UI controls
output_area = widgets.Output()

recommendation_display = widgets.HTML(
    value="<b><font color='blue'>🤖 Getting recommendation...</font></b>"
)

def show_recommendation():
    try:
        rec = recommender_agent.get_agent_action_str()
        recommendation_display.value = f"<b><font color='red'>🤖 AI: {rec}</font></b>"
    except Exception as e:
        recommendation_display.value = f"<b><font color='orange'>⚠️ Error: {e}</font></b>"

if 'recommender_agent' in locals():
    show_recommendation()

print("✅ UI created!")

In [ ]:
# Action controls
action_type = widgets.Dropdown(
    options=['Group', 'Filter', 'Back'],
    description='Action:'
)

column_select = widgets.Dropdown(
    options=['packet_number', 'eth_src', 'eth_dst', 'tcp_srcport', 'tcp_dstport'],
    description='Column:'
)

apply_ai_btn = widgets.Button(
    description='✅ Apply AI Recommendation',
    button_style='success'
)

apply_custom_btn = widgets.Button(
    description='🎯 Apply My Action',
    button_style='info'
)

reset_btn = widgets.Button(
    description='🔄 Reset',
    button_style='warning'
)

print("✅ Controls ready!")

In [ ]:
# Button handlers
def apply_ai_handler(btn):
    with output_area:
        clear_output(wait=True)
        try:
            result = recommender_agent.apply_agent_action(use_last_recommendation=True)
            if result.df_to_display is not None:
                print(f"✅ Result ({len(result.df_to_display)} rows):")
                display(result.df_to_display.head(10))
        except Exception as e:
            print(f"❌ Error: {e}")
    show_recommendation()

def reset_handler(btn):
    with output_area:
        clear_output(wait=True)
        try:
            recommender_agent.reset_environment()
            print("✅ Reset complete!")
            display(recommender_agent.original_dataset.head(10))
        except Exception as e:
            print(f"❌ Error: {e}")
    show_recommendation()

apply_ai_btn.on_click(apply_ai_handler)
reset_btn.on_click(reset_handler)

print("✅ Handlers ready!")

## 🎮 Live Interface

In [ ]:
# Display interface
ui = widgets.VBox([
    widgets.HTML("<h3>🎛️ Control Panel</h3>"),
    recommendation_display,
    widgets.HTML("<hr>"),
    widgets.HBox([action_type, column_select]),
    widgets.HBox([apply_ai_btn, apply_custom_btn, reset_btn]),
    widgets.HTML("<hr>"),
    widgets.HTML("<h3>📊 Results</h3>"),
    output_area
])

display(ui)

# Show initial data
if 'recommender_agent' in locals():
    with output_area:
        print("🗂️ Original Dataset:")
        display(recommender_agent.original_dataset.head(10))

---

## 🎉 Success!

### Usage:
1. **AI Recommendation** shown above
2. **Click buttons** to apply actions
3. **See results** in output area
4. **Repeat** to explore!

### Tips:
- Follow AI to see what it learned
- Try your own actions
- Use Reset to start over

**Enjoy exploring with AI! 🚀**